In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
from torch_geometric.datasets import Planetoid
dataset = Planetoid(root='/data/CiteSeer', name='CiteSeer')
print(len(dataset))
data = dataset[0]
data = data.to(device)
print(data)

cuda
1
Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [2]:
import torch.nn.functional as F
import nni.retiarii.nn.pytorch as nn
import torch_geometric.nn as gnn
import nni
from nni.retiarii import model_wrapper

```python
class DepthwiseSeparableConv(nn.Module):
    def __init__(self, in_ch, out_ch):
        super().__init__()
        self.depthwise = nn.Conv2d(in_ch, in_ch, kernel_size=3, groups=in_ch)
        self.pointwise = nn.Conv2d(in_ch, out_ch, kernel_size=1)

    def forward(self, x):
        return self.pointwise(self.depthwise(x))


@model_wrapper
class ModelSpace(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        # LayerChoice is used to select a layer between Conv2d and DwConv.
        self.conv2 = nn.LayerChoice([
            nn.Conv2d(32, 64, 3, 1),
            DepthwiseSeparableConv(32, 64)
        ])
        # ValueChoice is used to select a dropout rate.
        # ValueChoice can be used as parameter of modules wrapped in `nni.retiarii.nn.pytorch`
        # or customized modules wrapped with `@basic_unit`.
        self.dropout1 = nn.Dropout(nn.ValueChoice([0.25, 0.5, 0.75]))  # choose dropout rate from 0.25, 0.5 and 0.75
        self.dropout2 = nn.Dropout(0.5)
        feature = nn.ValueChoice([64, 128, 256])
        self.fc1 = nn.Linear(9216, feature)
        self.fc2 = nn.Linear(feature, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(self.conv2(x), 2)
        x = torch.flatten(self.dropout1(x), 1)
        x = self.fc2(self.dropout2(F.relu(self.fc1(x))))
        output = F.log_softmax(x, dim=1)
        return output
model_space = ModelSpace()
print(model_space)

```

In [3]:
@model_wrapper
class MySpace(nn.Module):
    def __init__(self, num_node_features, num_classes, hidden=20):
        super().__init__()
        #self.conv1 = GCNConv(num_node_features, 16)
        #self.conv2 = GCNConv(16, num_classes)
        self.conv1 = nn.LayerChoice([
            #gnn.GATconv(num_node_features, hidden),
            gnn.GCNConv(num_node_features, hidden),
            #gnn.GINconv(num_node_features, hidden),
            gnn.ChebConv(num_node_features, hidden, K=2),
            gnn.SAGEConv(num_node_features, hidden)
        ])
        self.dropout1 = nn.Dropout(nn.ValueChoice([0.1, 0.25, 0.5, 0.75]))  # choose dropout rate from 0.25, 0.5 and 0.75
        self.conv2 = nn.LayerChoice([
            #gnn.GATconv(hidden, num_classes),
            gnn.GCNConv(hidden, num_classes),
            #gnn.GINconv(hidden, num_classes),
            gnn.ChebConv(hidden, num_classes, K=2),
            gnn.SAGEConv(hidden, num_classes)
        ])
        self.dropout2 = nn.Dropout(0.5)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.dropout1(x, training=self.training)
        x = self.conv2(x, edge_index)
        x = F.relu(x)
        x = self.dropout2(x, training=self.training)
        x = F.softmax(x, dim=1)

        return x
    

In [4]:
my_space = MySpace(dataset.num_node_features, dataset.num_classes)
print(my_space)

MySpace(
  (conv1): LayerChoice([GCNConv(3703, 20), ChebConv(3703, 20, K=2, normalization=sym), SAGEConv(3703, 20, aggr=mean)], label='model_1')
  (dropout1): Dropout(p=0.1, inplace=False)
  (conv2): LayerChoice([GCNConv(20, 6), ChebConv(20, 6, K=2, normalization=sym), SAGEConv(20, 6, aggr=mean)], label='model_3')
  (dropout2): Dropout(p=0.5, inplace=False)
)


In [5]:
import nni.retiarii.strategy as strategy
search_strategy = strategy.Random(dedup=True)  # dedup=False if deduplication is not wanted

In [6]:
def eval_model(model, myprint=False):
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)
    loss_function = torch.nn.CrossEntropyLoss().to(device)
    model.train()
    for epoch in range(100):
        optimizer.zero_grad()
        out = model(data)
        loss = loss_function(out[data.train_mask], data.y[data.train_mask])
        loss.backward()
        optimizer.step()
        if myprint and epoch % 20 == 0:
            print("epoch {}, loss = {}".format(epoch, loss))
    model.eval()
    pred = model(data).argmax(dim=1)
    correct = (pred[data.val_mask] == data.y[data.val_mask]).sum()
    acc = int(correct) / int(data.val_mask.sum())
    if myprint:
        print(f'Accuracy: {acc:.4f}')
    #return acc
    nni.report_final_result(acc)

In [7]:
from nni.retiarii.evaluator import FunctionalEvaluator
evaluator = FunctionalEvaluator(eval_model)

In [8]:
from nni.retiarii.experiment.pytorch import RetiariiExperiment, RetiariiExeConfig
exp = RetiariiExperiment(my_space, evaluator, [], search_strategy)
exp_config = RetiariiExeConfig('local')
exp_config.experiment_name = 'mnist_search'
exp_config.max_trial_number = 20   # spawn 4 trials at most
exp_config.trial_concurrency = 1  # will run two trials concurrently

exp_config.trial_gpu_number = 1
exp_config.training_service.use_active_gpu = True
'''
from nni.nas.experiment import NasExperiment
exp = NasExperiment(my_space, evaluator, search_strategy)
exp.config.max_trial_number = 20   # spawn 4 trials at most
exp.config.trial_concurrency = 1

exp.config.trial_gpu_number = 1
exp.config.training_service.use_active_gpu = True
'''

'\nfrom nni.nas.experiment import NasExperiment\nexp = NasExperiment(my_space, evaluator, search_strategy)\nexp.config.max_trial_number = 20   # spawn 4 trials at most\nexp.config.trial_concurrency = 1\n\nexp.config.trial_gpu_number = 1\nexp.config.training_service.use_active_gpu = True\n'

In [9]:
exp.run(exp_config, 8081)

[2023-06-04 23:04:14] Creating experiment, Experiment ID: edm9f0yq
[2023-06-04 23:04:14] Starting web server...
[2023-06-04 23:04:16] Setting up...
[2023-06-04 23:04:16] Web portal URLs: http://169.254.189.172:8081 http://169.254.239.17:8081 http://169.254.159.216:8081 http://172.20.3.190:8081 http://169.254.29.87:8081 http://169.254.244.73:8081 http://127.0.0.1:8081
[2023-06-04 23:04:16] Dispatcher started
[2023-06-04 23:04:17] Start strategy...
[2023-06-04 23:04:17] Successfully update searchSpace.
[2023-06-04 23:04:17] Random search running in fixed size mode. Dedup: on.
[2023-06-04 23:15:08] WARNING: KeyboardInterrupt detected
[2023-06-04 23:15:08] Stopping experiment, please wait...
[2023-06-04 23:15:08] Dispatcher exiting...
[2023-06-04 23:15:10] Dispatcher terminiated
[2023-06-04 23:15:10] Experiment stopped
[2023-06-04 23:15:10] Search process is done, the experiment is still alive, `stop()` can terminate the experiment.
